In [ ]:
import csv, re, random, math, pandas

In [ ]:
import json

In [ ]:
from torchmetrics import ConfusionMatrix

In [ ]:
with open("../FindTaskCorpus.csv") as f:
    reader = csv.reader(f)
    lines = list(reader)

In [ ]:
#print(len(lines))
corpus = [] #{trial_id: str, ot: str, o: str, init_action: str,
           # moves: [(actor, utterance, da, pt, ho, eldL, eldOtGiven, eldLGiven, helOtBelief, helLBelief, helOBelief, eldAction, eldSeesO)]}
i = 1
while i < len(lines):
    #print(i)
    trial_id = lines[i][0]
    ot = lines[i+1][0].lower()
    o = lines[i+1][1].lower()
    init_action = lines[i+1][11]
    init_sees = lines[i+1][12]
    #print(i, trial_id, ot, o)
    i += 2
    if not o: #skip to next trial
        while lines[i][0]:
            i += 1
        i += 1
        if i == 2322: #2381
            break
        continue
    moves = []
    while lines[i][0]:
        if lines[i][6]: #has been annotated
            move = lines[i]
            if move[0]=='hel' and move[11]=='': #eld_action not annotated
                move[11]=0
                #print(move)
            if move[0]=='hel' and move[13]=='': #hel_action not annotated
                print('*******************************************************************')
                print(move)
                if lines[i-1][0]=='hel':
                    print(lines[i-1])
                if lines[i+1][0]=='hel':
                    print(lines[i+1])
                    
                if (not lines[i-1][13] =='') or (not lines[i+1][13] ==''):
                    move[13] = lines[i-1][13]
                    if move[13] == '':
                        move[13] = lines[i+1][13]
                if move[13] == '':
                    move[13] = 6 #12
                print(move)
                    
            if not move[12]: #no eldSeesO annotation
                eld_sees = "0"
                if not move[4].startswith("Close"):
                    if moves:
                        eld_sees = moves[-1][12]
                    elif init_sees:
                        eld_sees = init_sees
                move[12] = eld_sees
            moves.append(tuple(move))
        i += 1
    corpus.append({"trial_id": trial_id, "ot": ot, "o": o, "init_action": init_action, "moves": moves})
    i += 1
    #print(i)


In [ ]:
condensed_corpus = []  # combine consecutive moves by same actor if state doesn't change
for trial in corpus:
    moves = (
        []
    )  # moves: [(actor, utterance, da, pt, ho, eldL, eldOtGiven, eldLGiven, helOtBelief, helLBelief, helOBelief, eldAction, eldSeesO)]
    for move in trial["moves"]:
        if len(moves) == 0:
            moves.append(move)
        else:
            prev_move = moves[-1]
            if prev_move[0] == move[0] and move[6:11] == prev_move[6:11]:
                # if move and prev_move both have pt/HO, consider as separate moves
                if (move[3] or move[4]) and (prev_move[3] or prev_move[4]):
                    moves.append(move)
                    continue
                prev_move = moves.pop(-1)
                new_move = []
                # actors should be the same
                new_move.append(move[0])
                # concatenate utterances
                new_move.append(" ".join([prev_move[1], move[1]]).strip())
                # take latest DA unless it's 'Explain'
                new_move.append(
                    prev_move[2]
                    if prev_move[2] and (not move[2] or move[2] == "Explain")
                    else move[2]
                )
                # take latest existing pt/ho
                new_move.append(
                    prev_move[3] if prev_move[3] and not move[3] else move[3]
                )
                new_move.append(
                    prev_move[4] if prev_move[4] and not move[4] else move[4]
                )
                # take latest L, eldAction, eldSeesO
                # states should be the same
                new_move.extend(move[5:])
                new_move = tuple(new_move)
                moves.append(new_move)
            else:
                moves.append(move)
    condensed_corpus.append(
        {
            "trial_id": trial["trial_id"],
            "ot": trial["ot"],
            "o": trial["o"],
            "init_action": trial["init_action"],
            "moves": moves,
        }
    )

In [ ]:
condensed_corpus

In [ ]:
LOC = {"bed", "c", "corner", "counter", "d", "freezer", "fridge", "sink", "stove", "table"}
PLURAL = {"bowls": "bowl", "dishes": "dish", "glasses": "glass", "grapes": "grape", 
          "napkins": "napkin", "plates": "plate", "pots": "pot"}
SINGULAR = {s:p for p,s in PLURAL.items()}
OBJ_SUB = {"bowl": "dish", "plate": "dish", "salad bowl": "bowl", 
           "glass": "cup", "grape": "fruit", "macaroni": "pasta",
           "bottle of oil": "bottle", "knife": "silverware",
           "fork": "silverware", "spoon": "silverware"}
OBJ_GROUPS = {"bowl": {"dish", "salad bowl"}, "cup": {"glass"},
              "dish": {"bowl", "plate", "salad bowl"}, "fruit": {"grape"},
              "glass": {"cup"}, "grape": {"fruit"}, "ice cream": {"bowl"},
              "juice": {"cup", "glass"}, "milk": {"cup", "glass"}, 
              "pasta": {"dish", "plate", "pot"}, "plate": {"dish"}, 
              "salad": {"bowl", "dish", "plate"}, "salad bowl": {"bowl", "dish"},
              "silverware": {"spoon"}, "spoon": {"silverware"}}

data = []
for trial in condensed_corpus:
    for i,move in enumerate(trial["moves"]):
        if move[0] == "eld": continue
            
        next_actor = trial["moves"][i+1][0] if i+1 <len(trial["moves"]) else ""
        #if next_actor == "hel" or next_actor == "": continue
        #if next_actor == "": continue

        datapoint = {}
        datapoint["id"] = "{}|{}".format(trial["trial_id"], i+1)
        datapoint["ot"] = trial["ot"]

        utt = move[1].lower() if move[1] else "action"
        utt = re.sub("\s\s+", " ", re.sub("[.?!,]|xxx", "", utt))
        datapoint["utt"] = utt
        datapoint["da"] = move[2]
        datapoint["action"] = int(move[13])
        datapoint["actor"] = move[0]
        datapoint["next_actor"] = next_actor   

        if i == 0:
            prev_utt = "start"
        else:
            if trial["moves"][i-1][1]:
                prev_utt = trial["moves"][i-1][1].lower()
            else:
                prev_utt = "action"
        prev_utt = re.sub("\s\s+", " ", re.sub("[.?!,]|xxx", "", prev_utt))
        datapoint["prev_utt"] = prev_utt
        datapoint["prev_da"] = trial["moves"][i-1][2] if i > 0 else ""

        datapoint["prev_actor"] = trial["moves"][i-1][0] if i > 0 else ""
        datapoint["prev_ot_given"] = int(trial["moves"][i-1][6]) if i > 0 else 0
        datapoint["prev_l_given"] = int(trial["moves"][i-1][7]) if i > 0 else 0
        datapoint["prev_hel_ot_belief"] = int(trial["moves"][i-1][8]) if i > 0 else 0
        datapoint["prev_hel_l_belief"] = int(trial["moves"][i-1][9]) if i > 0 else 0
        datapoint["prev_hel_o_belief"] = int(trial["moves"][i-1][10]) if i > 0 else 0
        
        eld_ot = PLURAL.get(trial["ot"], trial["ot"])
        eld_l = re.sub(' *, *', ',', move[5].lower()).strip('[]').split(',')
        eld_o = trial["o"].lower()
        datapoint["o"] = eld_o

        pt_target = 0
        pt = 0
        if move[3]:
            targets = re.sub(' *, *', ',', move[3].lower()).strip('[]').split(',')
            target_name = targets[0].split('#')[0] #check first target for OBJ or LOC
            if target_name in LOC:
                pt_target = 1
                if eld_l[0] == '?' or targets[0] in eld_l:
                    pt = 1
                else:
                    pt = 2
            else: #target is OBJ
                pt_target = 2
                if eld_o in targets:
                    pt = 1
                else:
                    target_name = PLURAL.get(target_name, target_name)
                    if OBJ_SUB.get(target_name, target_name) == OBJ_SUB.get(eld_ot, eld_ot):
                        pt = 2
                    else:
                        pt = 3
        datapoint["pt_target"] = pt_target
        datapoint["pt"] = pt

        ho_target = 0
        ho = 0
        ho_type = ""
        if move[4]:
            ho_type, targets = move[4].split(':')
            targets = re.sub(' *, *', ',', targets.lower()).strip('[]').split(',')
            target_name = targets[0].split('#')[0] #check first target for OBJ or LOC
            if target_name in LOC:
                ho_target = 1
                if eld_l[0] == '?' or targets[0] in eld_l:
                    ho = 1
                else:
                    ho = 2
            else: #target is OBJ
                ho_target = 2
                if eld_o in targets:
                    ho = 1
                else:
                    target_name = PLURAL.get(target_name, target_name)
                    if OBJ_SUB.get(target_name, target_name) == OBJ_SUB.get(eld_ot, eld_ot):
                        ho = 2
                    else:
                        ho = 3
        datapoint["ho_target"] = ho_target
        datapoint["ho"] = ho
        datapoint["ho_type"] = ho_type

        for j in range(i-1, -1, -1):
            if trial["moves"][j][0] == 'hel':
                prev_eld_action = int(trial["moves"][j][11])
                break
        else:
            prev_eld_action = int(trial["init_action"])
        datapoint["prev_eld_action"] = prev_eld_action

        datapoint["hel_ot_belief"] = int(move[8])
        datapoint["hel_l_belief"] = int(move[9])
        datapoint["hel_o_belief"] = int(move[10])
        #print(move)
        datapoint["eld_action"] = int(move[11])
        if i+1<len(trial['moves']) and trial['moves'][i+1][0]== 'eld':
            datapoint["eld_da"]=trial['moves'][i+1][2]
        #elif i+2<len(trial['moves']) and trial['moves'][i+2][0]== 'eld':
            #datapoint["eld_da"]=trial['moves'][i+2][2]
        #elif i+3<len(trial['moves']) and trial['moves'][i+3][0]== 'eld':
            #datapoint["eld_da"]=trial['moves'][i+3][2]
        else:
            datapoint["eld_da"]=""
        datapoint["eld_sees_o"] = int(move[12])

        data.append(datapoint)

In [ ]:
data[0]

In [ ]:
data[13]

In [ ]:
data[15]

In [ ]:
for datapoint in data:
    if (datapoint['prev_hel_ot_belief']==1 and datapoint['prev_hel_l_belief']==0 
        and datapoint['prev_hel_o_belief']==0 and datapoint['prev_actor']=='eld'):
        print(datapoint)

In [ ]:
len(data)

In [ ]:
random.seed(0)
idxs = list(range(len(data)))
random.shuffle(idxs)
train_len = int(0.8*len(data))
valid_len = int(0.1*len(data))
train_idxs = idxs[:train_len]
valid_idxs = idxs[train_len:train_len+valid_len]
test_idxs = idxs[train_len+valid_len:]

In [ ]:
objs = []
spec_objs = []

for i in train_idxs:
    dp = data[i]
    for j,trial in enumerate(condensed_corpus):
        trial_id, move_id = dp["id"].split('|')
        if trial["trial_id"] == trial_id:
            trial_idx = j
            move_idx = int(move_id)-1
            break
    else:
        print("couldn't find datapoint", dp)
    
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and
        (dp["prev_actor"] == 'hel' and (re.search(dp['ot'], dp['utt']) or condensed_corpus[trial_idx]["moves"][move_idx-2][10] == '3'))):
        #print(dp["ot"])
        if dp["ot"] not in objs:
            objs.append(dp["ot"])
        if dp["o"] not in spec_objs:
            spec_objs.append(dp["o"])
            
            
for i in valid_idxs:
    dp = data[i]
    for j,trial in enumerate(condensed_corpus):
        trial_id, move_id = dp["id"].split('|')
        if trial["trial_id"] == trial_id:
            trial_idx = j
            move_idx = int(move_id)-1
            break
    else:
        print("couldn't find datapoint", dp)
    
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and
        (dp["prev_actor"] == 'hel' and (re.search(dp['ot'], dp['utt']) or condensed_corpus[trial_idx]["moves"][move_idx-2][10] == '3'))):
        #print(dp["ot"])
        if dp["ot"] not in objs:
            objs.append(dp["ot"])
        if dp["o"] not in spec_objs:
            spec_objs.append(dp["o"])
            
for i in test_idxs:
    dp = data[i]
    for j,trial in enumerate(condensed_corpus):
        trial_id, move_id = dp["id"].split('|')
        if trial["trial_id"] == trial_id:
            trial_idx = j
            move_idx = int(move_id)-1
            break
    else:
        print("couldn't find datapoint", dp)
    
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and
        (dp["prev_actor"] == 'hel' and (re.search(dp['ot'], dp['utt']) or condensed_corpus[trial_idx]["moves"][move_idx-2][10] == '3'))):
        #print(dp["ot"])
        if dp["ot"] not in objs:
            objs.append(dp["ot"])
        if dp["o"] not in spec_objs:
            spec_objs.append(dp["o"])

In [ ]:
objs

In [ ]:
spec_objs

In [ ]:
locs = ["bed", "c", "corner", "counter", "d", "freezer", "fridge", "sink", "stove", "table"]

In [ ]:
states = {(i,j,k):0 for i in range(3) for j in range(4) for k in range(4)}
for d in [data[i] for i in train_idxs]:
    states[(d["hel_ot_belief"], d["hel_l_belief"], d["hel_o_belief"])] += 1
states

In [ ]:
random.seed(345)
train_aug = []

for i in train_idxs:
    dp = data[i]
    for j,trial in enumerate(condensed_corpus):
        trial_id, move_id = dp["id"].split('|')
        if trial["trial_id"] == trial_id:
            trial_idx = j
            move_idx = int(move_id)-1
            break
    else:
        print("couldn't find datapoint", dp)
    
    # add (0,0,0) and (0,1,0) instances
    # take random HEL actions and replace utterance with a random (0,0,0)/(0,1,0) utterance
    det_ot_utts = ['uh what did you need',
                   'alright what are we looking for',
                   'what are we looking for',
                   'okay what am i looking for',
                   'what do you need to find first']
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and
        (dp["prev_actor"] == 'eld' or dp["prev_da"] not in ["State-y", "State-n"]) and
        (dp["prev_actor"] == 'eld' or "Open" in condensed_corpus[trial_idx]["moves"][move_idx-1][4]) and
        (move_idx < 2 or condensed_corpus[trial_idx]["moves"][move_idx-2][10] != '3')): # hel action is not check ot 
        hel_l_belief = dp["prev_l_given"] or dp["prev_hel_l_belief"]
        to_add = []
        det_ot_utts_copy = list(det_ot_utts)
        random.shuffle(det_ot_utts_copy)
        for j in range(2): #1
            if ((hel_l_belief and random.random() < 0.15) or 
                (not hel_l_belief and random.random() < 0.75)):
                if len(det_ot_utts_copy) == 0: continue
                dp_aug = dict(dp)
                dp_aug["id"] += ".{}".format(len(to_add)+1)
                dp_aug["utt"] = det_ot_utts_copy.pop(0)
                dp_aug["da"] = "Query-w"
                dp_aug["action"] = 1
                dp_aug["hel_ot_belief"] = 0
                dp_aug["hel_l_belief"] = hel_l_belief
                dp_aug["hel_o_belief"] = 0
                dp_aug["pt"] = 0
                dp_aug["pt_target"] = 0
                dp_aug["ho"] = 0
                dp_aug["ho_target"] = 0
                dp_aug["ho_type"] = ''
                dp_aug["eld_action"] = 1
                dp_aug["eld_da"] = "Reply-w"
                dp_aug["next_actor"] = "eld"
                to_add.append(dp_aug)
        train_aug.extend(to_add)
        

    # add (2,0,0) and (2,1,0) instances
    # take random HEL actions containing the target OT and replace OT in utterance with a random different OT (that isn’t a sub/superclass)
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and dp["prev_actor"] == 'hel' 
        and ((re.search(dp['ot'], dp['utt']) or condensed_corpus[trial_idx]["moves"][move_idx-2][10] == '2'))):
        #print(dp)
        hel_l_belief = dp["prev_l_given"] or dp["prev_hel_l_belief"]
        x = objs[:]
        x.remove(dp["ot"])
        to_add = []
        for j in range(50): #50, 100, 150
            if ((hel_l_belief and random.random() < 0.02) or 
                (not hel_l_belief and random.random() < 0.9)):
                dp_aug = dict(dp)
                dp_aug["id"] += ".{}".format(len(to_add)+1)
                if len(x) >0:
                    #ot_aug = objs[random.randint(0, len(x)-1)]
                    ot_aug = x[random.randint(0, len(x)-1)]
                    x.remove(ot_aug)
                    dp_aug["utt"] = dp["utt"].replace(dp["ot"], ot_aug)
                    dp_aug["da"] = dp["da"]
                    dp_aug["action"] = dp["action"]
                    dp_aug["hel_ot_belief"] = 2
                    dp_aug["hel_l_belief"] = hel_l_belief
                    dp_aug["hel_o_belief"] = 0
                    dp_aug["pt"] = dp["pt"]
                    if dp_aug["pt_target"]==2:
                        dp_aug["pt"] = 2 #dp["pt"]
                    dp_aug["pt_target"] = dp["pt_target"]
                    dp_aug["ho"] = dp["ho"]
                    if dp_aug["ho_target"]==2:
                        dp_aug["ho"] = 2 #dp["pt"]
                    dp_aug["ho_target"] = dp["ho_target"]
                    dp_aug["ho_type"] = dp["ho_type"]
                    dp_aug["eld_action"] = 1
                    dp_aug["eld_da"] = "Instruct"
                    dp_aug["next_actor"] = "eld"
                    if not dp_aug in to_add:
                        to_add.append(dp_aug)
                #print(dp_aug)
        #print(to_add)
        train_aug.extend(to_add)
        
    ## add (1,1,2) instances
    ## randomly replace O in pointing and HO actions with a different O
    if (move_idx > 0 and dp["prev_actor"] == 'hel' and dp["hel_ot_belief"]==1 and dp["hel_l_belief"]==1 
        and (not dp["hel_o_belief"] == 0) and ((dp['pt'] and dp['pt_target']==2) or (dp['ho'] and dp['ho_target']==2))):
        #print(dp)
        x = spec_objs[:]
        if dp["o"] in x: 
            x.remove(dp["o"])
        to_add=[]
        for j in range(3): #2 #3, 5
            dp_aug = dict(dp)
            dp_aug["id"] += ".{}".format(len(to_add)+1)
            if len(x) >0:
                dp_aug["utt"] = dp["utt"]
                o_aug = x[random.randint(0, len(x)-1)]
                x.remove(o_aug)
                if dp["utt"]:
                    dp_aug["utt"] = dp["utt"].replace(dp["ot"], o_aug)
                    dp_aug["utt"] = dp["utt"].replace(dp["o"], o_aug)
                dp_aug["da"] = dp["da"]
                dp_aug["action"] = dp["action"]
                dp_aug["hel_ot_belief"] = 1
                dp_aug["hel_l_belief"] = 1
                dp_aug["hel_o_belief"] = 2
                dp_aug["pt"] = dp["pt"]
                if dp_aug["pt_target"]==2:
                    dp_aug["pt"] = 3 #dp["pt"]
                dp_aug["pt_target"] = dp["pt_target"]
                dp_aug["ho"] = dp["ho"]
                if dp_aug["ho_target"]==2:
                    dp_aug["ho"] = 3 #dp["pt"]
                dp_aug["ho_target"] = dp["ho_target"]
                dp_aug["ho_type"] = dp["ho_type"]
                dp_aug["eld_action"] = 1 #gives spec ot
                dp_aug["eld_da"] = "Instruct"
                dp_aug["next_actor"] = "eld"
                if not dp_aug in to_add:
                    to_add.append(dp_aug)
              #print(dp_aug)
          #print(to_add)
        train_aug.extend(to_add)
            
      # add (1,2,0) instances
      # randomly replace L in pointing and HO actions with a different L
    if (move_idx > 0 and dp["prev_actor"] == 'hel' and dp["hel_ot_belief"]==1 and dp["hel_l_belief"] in [1,3] 
        and dp["hel_o_belief"] == 0 and ((dp['pt'] and dp['pt_target']==1) or (dp['ho'] and dp['ho_target']==1))):
        #print(dp)
        to_add=[]
        for j in range(3): #2, 5, 1
            dp_aug = dict(dp)
            dp_aug["id"] += ".{}".format(len(to_add)+1)
            dp_aug["utt"] = dp["utt"]
            dp_aug["da"] = dp["da"]
            dp_aug["action"] = dp["action"]
            dp_aug["hel_ot_belief"] = 1
            dp_aug["hel_l_belief"] = 2
            dp_aug["hel_o_belief"] = 0
            dp_aug["pt"] = dp["pt"]
            if dp_aug["pt_target"]==1:
                dp_aug["pt"] = 2 #dp["pt"]
            dp_aug["pt_target"] = dp["pt_target"]
            dp_aug["ho"] = dp["ho"]
            if dp_aug["ho_target"]==1:
                dp_aug["ho"] = 2 #dp["pt"]
            dp_aug["ho_target"] = dp["ho_target"]
            dp_aug["ho_type"] = dp["ho_type"]
            dp_aug["eld_da"] = "Instruct"
            dp_aug["next_actor"] = "eld"
            dp_aug["eld_action"] = 2
            '''e = random.random()
            if e >= 0 and e< 0.6: #(0,0.5), (0,0.7)
                dp_aug["eld_action"] = 2 #gives l
            #elif e>= 0.25 and e< 0.5:
                #dp_aug["eld_action"] = 4 #gives l,ot
            if e>= 0.8 and e <= 1:
                dp_aug["eld_action"] = 2 #gives l 
            #elif e>= 0.75 and e<= 1:
                #dp_aug["eld_action"] = 5 #gives l,ot'''
            if not dp_aug in to_add:
                to_add.append(dp_aug)
            #print(dp_aug)
        #print(to_add)
        train_aug.extend(to_add)
        
    prev_utts = ['i need the ',
                'i am looking for the ',
                'get me the ',
                'bring me the ',
                'i said i wanted the ',
                'i said cabinet number 1',
                'look inside the drawer',
                'check c#2',
                'it should be inside d#4',
                'please check inside d#2']
    #(2,0,0) for prev state
    eps = random.random()
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==0 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=1
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
            
        train_aug.append(dp_aug)
    #(2,1,0) for prev state  
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=1
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
        train_aug.append(dp_aug)
    #(1,2,0) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=2
        dp_aug['prev_eld_action']=2
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[5]
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[6]
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[7]
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[8]
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[9]
        train_aug.append(dp_aug)
    #(2,2,0) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=2
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=3
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot'] + ". " + prev_utts[5]
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot'] + ". " + prev_utts[6]
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot'] + ". " + prev_utts[7]
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot'] + ". " + prev_utts[8]
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot'] + ". " + prev_utts[9]
        train_aug.append(dp_aug)
    #(1,1,2) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==1 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=1
        dp_aug['prev_hel_ot_belief']=1
        dp_aug['prev_hel_o_belief']=2
        dp_aug['prev_eld_action']=1
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
        dp_aug['pev_da']='Instruct'
        train_aug.append(dp_aug)    
        

In [ ]:
random.seed(345)
valid_aug = []

for i in valid_idxs:
    dp = data[i]
    for j,trial in enumerate(condensed_corpus):
        trial_id, move_id = dp["id"].split('|')
        if trial["trial_id"] == trial_id:
            trial_idx = j
            move_idx = int(move_id)-1
            break
    else:
        print("couldn't find datapoint", dp)
    
    # add (0,0,0) and (0,1,0) instances
    det_ot_utts = ['uh what did you need',
                   'alright what are we looking for',
                   'what are we looking for',
                   'okay what am i looking for',
                   'what do you need to find first']
    #print(trial_idx, move_idx)
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and
        (dp["prev_actor"] == 'eld' or dp["prev_da"] not in ["State-y", "State-n"]) and
        (dp["prev_actor"] == 'eld' or "Open" in condensed_corpus[trial_idx]["moves"][move_idx-1][4]) and
        (move_idx < 2 or condensed_corpus[trial_idx]["moves"][move_idx-2][10] != '3')):
        hel_l_belief = dp["prev_l_given"] or dp["prev_hel_l_belief"]
        to_add = []
        det_ot_utts_copy = list(det_ot_utts)
        random.shuffle(det_ot_utts_copy)
        for j in range(2):
            if ((hel_l_belief and random.random() < 0.15) or 
                (not hel_l_belief and random.random() < 0.75)):
                if len(det_ot_utts_copy) == 0: continue
                dp_aug = dict(dp)
                dp_aug["id"] += ".{}".format(len(to_add)+1)
                dp_aug["utt"] = det_ot_utts_copy.pop(0)
                dp_aug["da"] = "Query-w"
                dp_aug["action"] = 1
                dp_aug["hel_ot_belief"] = 0
                dp_aug["hel_l_belief"] = hel_l_belief
                dp_aug["hel_o_belief"] = 0
                dp_aug["pt"] = 0
                dp_aug["pt_target"] = 0
                dp_aug["ho"] = 0
                dp_aug["ho_target"] = 0
                dp_aug["ho_type"] = ''
                dp_aug["eld_action"] = 1
                dp_aug["eld_da"] = "Reply-w"
                dp_aug["next_actor"] = "eld"
                to_add.append(dp_aug)
        valid_aug.extend(to_add)
        

        # add (2,0,0) and (2,1,0) instances
        # take random HEL actions containing the target OT and replace OT in utterance with a random different OT (that isn’t a sub/superclass)
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and dp["prev_actor"] == 'hel' 
        and ((re.search(dp['ot'], dp['utt']) or condensed_corpus[trial_idx]["moves"][move_idx-2][10] == '2'))):
        
        #print(dp)
        hel_l_belief = dp["prev_l_given"] or dp["prev_hel_l_belief"]
        x = objs[:]
        #print(dp["ot"])
        #print(x)
        x.remove(dp["ot"])
        to_add = []
        for j in range(50): #50, 100, 150
            if ((hel_l_belief and random.random() < 0.02) or 
                (not hel_l_belief and random.random() < 0.9)):
                dp_aug = dict(dp)
                dp_aug["id"] += ".{}".format(0)
                if len(x) >0:
                    ot_aug = x[random.randint(0, len(x)-1)]
                    x.remove(ot_aug)
                    dp_aug["utt"] = dp["utt"].replace(dp["ot"], ot_aug)
                    dp_aug["da"] = dp["da"]
                    dp_aug["action"] = dp_aug["action"]
                    dp_aug["hel_ot_belief"] = 2
                    dp_aug["hel_l_belief"] = hel_l_belief
                    dp_aug["hel_o_belief"] = 0
                    dp_aug["pt"] = dp["pt"]
                    if dp_aug["pt_target"]==2:
                        dp_aug["pt"] = 2 #dp["pt"]
                    dp_aug["pt_target"] = dp["pt_target"]
                    dp_aug["ho"] = dp["ho"]
                    if dp_aug["ho_target"]==2:
                        dp_aug["ho"] = 2 #dp["pt"]
                    dp_aug["ho_target"] = dp["ho_target"]
                    dp_aug["ho_type"] = dp["ho_type"]
                    dp_aug["eld_action"] = 1
                    dp_aug["eld_da"] = "Instruct"
                    dp_aug["next_actor"] = "eld"
                    if not dp_aug in to_add:
                        to_add.append(dp_aug)
                #print(dp_aug)
        #print(to_add)
        valid_aug.extend(to_add)    
        # add (1,1,2) instances
    # randomly replace O in pointing and HO actions with a different O
    if (move_idx > 0 and dp["prev_actor"] == 'hel' and dp["hel_ot_belief"]==1 and dp["hel_l_belief"]==1 
        and (not dp["hel_o_belief"] == 0) and ((dp['pt'] and dp['pt_target']==2) or (dp['ho'] and dp['ho_target']==2))):
        #print(dp)
        to_add=[]
        x = spec_objs[:]
        if dp["o"] in x: 
            x.remove(dp["o"])
        to_add=[]
        for j in range(3): #2 #3, 5
            dp_aug = dict(dp)
            dp_aug["id"] += ".{}".format(0)
            if len(x) >0:
                dp_aug["utt"] = dp["utt"]
                o_aug = x[random.randint(0, len(x)-1)]
                x.remove(o_aug)
                if dp["utt"]:
                    dp_aug["utt"] = dp["utt"].replace(dp["ot"], o_aug)
                    dp_aug["utt"] = dp["utt"].replace(dp["o"], o_aug)
                dp_aug["da"] = dp["da"]
                dp_aug["action"] = dp["action"]
                dp_aug["hel_ot_belief"] = 1
                dp_aug["hel_l_belief"] = 1
                dp_aug["hel_o_belief"] = 2
                dp_aug["pt"] = dp["pt"]
                if dp_aug["pt_target"]==2:
                    dp_aug["pt"] = 3 #dp["pt"]
                dp_aug["pt_target"] = dp["pt_target"]
                dp_aug["ho"] = dp["ho"]
                if dp_aug["ho_target"]==2:
                    dp_aug["ho"] = 3 #dp["pt"]
                dp_aug["ho_target"] = dp["ho_target"]
                dp_aug["ho_type"] = dp["ho_type"]
                dp_aug["eld_action"] = 1 #gives spec ot
                dp_aug["eld_da"] = "Instruct"
                dp_aug["next_actor"] = "eld"
                if not dp_aug in to_add:
                    to_add.append(dp_aug)
            #print(dp_aug)
        #print(to_add)
        valid_aug.extend(to_add)    # add (1,2,0) instances
    # randomly replace L in pointing and HO actions with a different L
    if (move_idx > 0 and dp["prev_actor"] == 'hel' and dp["hel_ot_belief"]==1 and dp["hel_l_belief"] in [1,3] 
        and dp["hel_o_belief"] == 0 and ((dp['pt'] and dp['pt_target']==1) or (dp['ho'] and dp['ho_target']==1))):
        #print(dp)
        to_add=[]
        for j in range(3): #2, 5, 1
            dp_aug = dict(dp)
            dp_aug["id"] += ".{}".format(0)
            dp_aug["utt"] = dp["utt"]
            dp_aug["da"] = dp["da"]
            dp_aug["action"] = dp_aug["action"]
            dp_aug["hel_ot_belief"] = 1
            dp_aug["hel_l_belief"] = 2
            dp_aug["hel_o_belief"] = 0
            dp_aug["pt"] = dp["pt"]
            if dp_aug["pt_target"]==1:
                dp_aug["pt"] = 2 #dp["pt"]
            dp_aug["pt_target"] = dp["pt_target"]
            dp_aug["ho"] = dp["ho"]
            if dp_aug["ho_target"]==1:
                dp_aug["ho"] = 2 #dp["pt"]
            dp_aug["ho_target"] = dp["ho_target"]
            dp_aug["ho_type"] = dp["ho_type"]
            dp_aug["eld_da"] = "Instruct"
            dp_aug["next_actor"] = "eld"
            dp_aug["eld_action"] = 2
            '''e = random.random()
            if e >= 0 and e< 0.6: #(0,0.5), (0,0.7)
                dp_aug["eld_action"] = 2 #gives l
            #elif e>= 0.25 and e< 0.5:
                #dp_aug["eld_action"] = 4 #gives l,ot
            if e>= 0.8 and e <= 1:
                dp_aug["eld_action"] = 2 #gives l 
            #elif e>= 0.75 and e<= 1:
                #dp_aug["eld_action"] = 5 #gives l,ot'''
            if not dp_aug in to_add:
                to_add.append(dp_aug)
            #print(dp_aug)
        #print(to_add)
        valid_aug.extend(to_add)
        
    prev_utts = ['i need the ',
                'i am looking for the ',
                'get me the ',
                'bring me the ',
                'i said i wanted the ',
                'i said cabinet number 1',
                'look inside the drawer',
                'check c#2',
                'it should be inside d#4',
                'please check inside d#2']
    #(2,0,0) for prev state
    eps = random.random()
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==0 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=1
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
            
        valid_aug.append(dp_aug)
    #(2,1,0) for prev state  
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=1
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
        valid_aug.append(dp_aug)
    #(1,2,0) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=2
        dp_aug['prev_eld_action']=2
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[5]
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[6]
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[7]
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[8]
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[9]
        valid_aug.append(dp_aug)
    #(2,2,0) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=2
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=3
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot'] + ". " + prev_utts[5]
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot'] + ". " + prev_utts[6]
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot'] + ". " + prev_utts[7]
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot'] + ". " + prev_utts[8]
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot'] + ". " + prev_utts[9]
        valid_aug.append(dp_aug)
    #(1,1,2) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==1 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=1
        dp_aug['prev_hel_ot_belief']=1
        dp_aug['prev_hel_o_belief']=2
        dp_aug['prev_eld_action']=1
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
        dp_aug['pev_da']='Instruct'
        valid_aug.append(dp_aug)    
        

In [ ]:
random.seed(345)
test_aug = []

for i in test_idxs:
    dp = data[i]
    for j,trial in enumerate(condensed_corpus):
        trial_id, move_id = dp["id"].split('|')
        if trial["trial_id"] == trial_id:
            trial_idx = j
            move_idx = int(move_id)-1
            break
    else:
        print("couldn't find datapoint", dp)
    
    # add (0,0,0) and (0,1,0) instances
    det_ot_utts = ['uh what did you need',
                   'alright what are we looking for',
                   'what are we looking for',
                   'okay what am i looking for',
                   'what do you need to find first']
    #print(trial_idx, move_idx)
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and
        (dp["prev_actor"] == 'eld' or dp["prev_da"] not in ["State-y", "State-n"]) and
        (dp["prev_actor"] == 'eld' or "Open" in condensed_corpus[trial_idx]["moves"][move_idx-1][4]) and
        (move_idx < 2 or condensed_corpus[trial_idx]["moves"][move_idx-2][10] != '3')):
        hel_l_belief = dp["prev_l_given"] or dp["prev_hel_l_belief"]
        to_add = []
        det_ot_utts_copy = list(det_ot_utts)
        random.shuffle(det_ot_utts_copy)
        for j in range(2):
            if ((hel_l_belief and random.random() < 0.15) or 
                (not hel_l_belief and random.random() < 0.75)):
                if len(det_ot_utts_copy) == 0: continue
                dp_aug = dict(dp)
                dp_aug["id"] += ".{}".format(0) #".{}".format(len(to_add)+1)
                dp_aug["utt"] = det_ot_utts_copy.pop(0)
                dp_aug["da"] = "Query-w"
                dp_aug["action"] = 1
                dp_aug["hel_ot_belief"] = 0
                dp_aug["hel_l_belief"] = hel_l_belief
                dp_aug["hel_o_belief"] = 0
                dp_aug["pt"] = 0
                dp_aug["pt_target"] = 0
                dp_aug["ho"] = 0
                dp_aug["ho_target"] = 0
                dp_aug["ho_type"] = ''
                dp_aug["eld_action"] = 1
                dp_aug["eld_da"] = "Reply-w"
                dp_aug["next_actor"] = "eld"
                to_add.append(dp_aug)
        test_aug.extend(to_add)
        

        # add (2,0,0) and (2,1,0) instances
        # take random HEL actions containing the target OT and replace OT in utterance with a random different OT (that isn’t a sub/superclass)
    if (move_idx > 0 and
        dp["prev_hel_o_belief"] not in [1, 3] and dp["prev_actor"] == 'hel' 
        and ((re.search(dp['ot'], dp['utt']) or condensed_corpus[trial_idx]["moves"][move_idx-2][10] == '2'))):
        
        #print(dp)
        hel_l_belief = dp["prev_l_given"] or dp["prev_hel_l_belief"]
        x = objs[:]
        #print(dp["ot"])
        #print(x)
        x.remove(dp["ot"])
        
        to_add = []
        for j in range(50): #50, 100, 150
            if ((hel_l_belief and random.random() < 0.02) or 
                (not hel_l_belief and random.random() < 0.9)):
                dp_aug = dict(dp)
                dp_aug["id"] += ".{}".format(0)#".{}".format(len(to_add)+1)
                if len(x) >0:
                    ot_aug = x[random.randint(0, len(x)-1)]
                    x.remove(ot_aug)
                    #print(ot_aug)
                    #print(x)
                
                    dp_aug["utt"] = dp["utt"].replace(dp["ot"], ot_aug)
                    dp_aug["da"] = dp["da"]
                    dp_aug["action"] = dp_aug["action"]
                    dp_aug["hel_ot_belief"] = 2
                    dp_aug["hel_l_belief"] = hel_l_belief
                    dp_aug["hel_o_belief"] = 0
                    dp_aug["pt"] = dp["pt"]
                    if dp_aug["pt_target"]==2:
                        dp_aug["pt"] = 2 #dp["pt"]
                    dp_aug["pt_target"] = dp["pt_target"]
                    dp_aug["ho"] = dp["ho"]
                    if dp_aug["ho_target"]==2:
                        dp_aug["ho"] = 2 #dp["pt"]
                    dp_aug["ho_target"] = dp["ho_target"]
                    dp_aug["ho_type"] = dp["ho_type"]
                    dp_aug["eld_action"] = 1
                    dp_aug["eld_da"] = "Instruct"
                    dp_aug["next_actor"] = "eld"
                    if not dp_aug in to_add:
                        to_add.append(dp_aug)
                #print(dp_aug)
        #print(to_add)
        test_aug.extend(to_add)    
    # add (1,1,2) instances
    # randomly replace O in pointing and HO actions with a different O
    if (move_idx > 0 and dp["prev_actor"] == 'hel' and dp["hel_ot_belief"]==1 and dp["hel_l_belief"]==1 
        and (not dp["hel_o_belief"] == 0) and ((dp['pt'] and dp['pt_target']==2) or (dp['ho'] and dp['ho_target']==2))):
        #print(dp)
        to_add=[]
        x = spec_objs[:]
        if dp["o"] in x: 
            x.remove(dp["o"])
        for j in range(3): #2 #3, 5
            dp_aug = dict(dp)
            dp_aug["id"] += ".{}".format(0)
            if len(x) >0:
                dp_aug["utt"] = dp["utt"]
                o_aug = x[random.randint(0, len(x)-1)]
                x.remove(o_aug)
                if dp["utt"]:
                    dp_aug["utt"] = dp["utt"].replace(dp["ot"], o_aug)
                    dp_aug["utt"] = dp["utt"].replace(dp["o"], o_aug)
                dp_aug["da"] = dp["da"]
                dp_aug["action"] = dp["action"]
                dp_aug["hel_ot_belief"] = 1
                dp_aug["hel_l_belief"] = 1
                dp_aug["hel_o_belief"] = 2
                dp_aug["pt"] = dp["pt"]
                if dp_aug["pt_target"]==2:
                    dp_aug["pt"] = 3 #dp["pt"]
                dp_aug["pt_target"] = dp["pt_target"]
                dp_aug["ho"] = dp["ho"]
                if dp_aug["ho_target"]==2:
                    dp_aug["ho"] = 3 #dp["pt"]
                dp_aug["ho_target"] = dp["ho_target"]
                dp_aug["ho_type"] = dp["ho_type"]
                dp_aug["eld_action"] = 1 #gives spec ot
                dp_aug["eld_da"] = "Instruct"
                dp_aug["next_actor"] = "eld"
                if not dp_aug in to_add:
                    to_add.append(dp_aug)
            #print(dp_aug)
        #print(to_add)
        test_aug.extend(to_add)    
        
    # add (1,2,0) instances
    # randomly replace L in pointing and HO actions with a different L
    if (move_idx > 0 and dp["prev_actor"] == 'hel' and dp["hel_ot_belief"]==1 and dp["hel_l_belief"] in [1,3] 
        and dp["hel_o_belief"] == 0 and ((dp['pt'] and dp['pt_target']==1) or (dp['ho'] and dp['ho_target']==1))):
        #print(dp)
        to_add=[]
        for j in range(3): #2, 5, 1
            dp_aug = dict(dp)
            dp_aug["id"] += ".{}".format(0) #dp_aug["id"] += ".{}".format(len(to_add)+1)
            dp_aug["utt"] = dp["utt"]
            dp_aug["da"] = dp["da"]
            dp_aug["action"] = dp_aug["action"]
            dp_aug["hel_ot_belief"] = 1
            dp_aug["hel_l_belief"] = 2
            dp_aug["hel_o_belief"] = 0
            dp_aug["pt"] = dp["pt"]
            if dp_aug["pt_target"]==1:
                dp_aug["pt"] = 2 #dp["pt"]
            dp_aug["pt_target"] = dp["pt_target"]
            dp_aug["ho"] = dp["ho"]
            if dp_aug["ho_target"]==1:
                dp_aug["ho"] = 2 #dp["pt"]
            dp_aug["ho_target"] = dp["ho_target"]
            dp_aug["ho_type"] = dp["ho_type"]
            dp_aug["eld_da"] = "Instruct"
            dp_aug["next_actor"] = "eld"
            dp_aug["eld_action"] = 2
            '''e = random.random()
            if e >= 0 and e< 0.6: #(0,0.5), (0,0.7)
                dp_aug["eld_action"] = 2 #gives l
            #elif e>= 0.25 and e< 0.5:
                #dp_aug["eld_action"] = 4 #gives l,ot
            if e>= 0.8 and e <= 1:
                dp_aug["eld_action"] = 2 #gives l 
            #elif e>= 0.75 and e<= 1:
                #dp_aug["eld_action"] = 5 #gives l,ot'''
            if not dp_aug in to_add:
                to_add.append(dp_aug)
            #print(dp_aug)
        #print(to_add)
        test_aug.extend(to_add)

    prev_utts = ['i need the ',
                'i am looking for the ',
                'get me the ',
                'bring me the ',
                'i said i wanted the ',
                'i said cabinet number 1',
                'look inside the drawer',
                'check c#2',
                'it should be inside d#4',
                'please check inside d#2']
    #(2,0,0) for prev state
    eps = random.random()
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==0 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=1
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
            
        test_aug.append(dp_aug)
    #(2,1,0) for prev state  
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=1
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
        test_aug.append(dp_aug)
    #(1,2,0) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=2
        dp_aug['prev_eld_action']=2
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[5]
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[6]
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[7]
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[8]
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[9]
        test_aug.append(dp_aug)
    #(2,2,0) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==0 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=2
        dp_aug['prev_hel_ot_belief']=2
        dp_aug['prev_eld_action']=3
        dp_aug['pev_da']='Instruct'
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot'] + ". " + prev_utts[5]
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot'] + ". " + prev_utts[6]
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot'] + ". " + prev_utts[7]
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot'] + ". " + prev_utts[8]
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot'] + ". " + prev_utts[9]
        test_aug.append(dp_aug)
    #(1,1,2) for prev state
    if (move_idx > 0 and dp['prev_hel_ot_belief']==1 and dp['prev_hel_l_belief']==1 
        and dp['prev_hel_o_belief']==1 and dp['prev_actor']=='eld'):
        dp_aug = dp.copy()
        dp_aug['prev_hel_l_belief']=1
        dp_aug['prev_hel_ot_belief']=1
        dp_aug['prev_hel_o_belief']=2
        dp_aug['prev_eld_action']=1
        if eps<0.2:
            dp_aug['prev_utt'] = prev_utts[0] + dp['ot']
        elif eps>=0.2 and eps<0.4:
            dp_aug['prev_utt'] = prev_utts[1] + dp['ot']
        elif eps>=0.4 and eps<0.6:
            dp_aug['prev_utt'] = prev_utts[2] + dp['ot']
        elif eps>=0.6 and eps<0.8:
            dp_aug['prev_utt'] = prev_utts[3] + dp['ot']
        elif eps>0.8:
            dp_aug['prev_utt'] = prev_utts[4] + dp['ot']
        dp_aug['pev_da']='Instruct'
        test_aug.append(dp_aug)    
        

In [ ]:
train_aug[2]

In [ ]:
states = {(i,j,k):0 for i in range(3) for j in range(4) for k in range(4)}
for d in [data[i] for i in train_idxs]+train_aug:
    states[(d["prev_hel_ot_belief"], d["prev_hel_l_belief"], d["prev_hel_o_belief"])] += 1
states

In [ ]:
from transformers import AdamW
import torch
import torch.nn as nn
import torch.nn.functional as F

from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
lang_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
device

In [ ]:
DA = {"": 0, "Instruct": 1, "Acknowledge": 2, "Query-w": 3, "Query-yn": 4, "Reply-w": 5, "Reply-y": 6, 
      "Reply-n": 7, "Check": 8, "Explain": 9, "Align": 10, "State-y": 11, "State-n": 12, "State": 13}

class DataHandler:
    # NOT for online evaluation!
    # data: list of dictionaries containing the above keys
    def __init__(self, data, lang_model, tokenizer, batch_size=64, shuffle=False):
        self._data = data
        self._n = len(data)
        self._model = lang_model
        self._tokenizer = tokenizer
        self._batch_size = batch_size
        self._shuffle = shuffle
        self._indices = list(range(self._n))
        if shuffle:
            random.shuffle(self._indices)
        self._num_steps = -(-self._n // self._batch_size)
        
    def generate_data(self):
        for i in range(self._num_steps):
            indices = self._indices[i*self._batch_size:(i+1)*self._batch_size]
            yield self._prepare_data(indices)

    def get_all_data(self):
        return self._prepare_data(self._indices)
    
    def encode_state(self, ot, l, o):
        if ot==0 and l==0 and o==0:
            return 0
        elif ot==0 and l==1 and o==0:
            return 1
        elif ot==0 and (l in [2,3]) and o==0:
            return 2
        elif ot==1 and l==0 and o==0:
            return 3
        elif ot==1 and l==0 and o==1:
            return 4
        elif ot==1 and l==0 and (o in [2,3]):
            return 5
        elif ot==1 and l==1 and o==0:
            return 6
        elif ot==1 and l==1 and o==1:
            return 7
        elif ot==1 and l==1 and (o in [2,3]):
            return 8
        elif ot==1 and (l in [2,3]) and o==0:
            return 9
        elif (ot in [2,3]) and l==0 and o==0:
            return 10
        elif (ot in [2,3]) and l==1 and o==0:
            return 11
        elif (ot in [2,3]) and (l in [2,3]) and o==0:
            return 12
        else:
            return None

    def _prepare_data(self, indices):
        #to_tokenize = [[self._data[i]["utt"], self._data[i]["prev_utt"]] for i in indices]
        #tokenized = self._tokenizer(to_tokenize, return_tensors="pt", padding=True)
        inputs = []
        
        for i in indices:
            this_in = []

            prev_actor = self._data[i]["prev_actor"]
            this_in.extend([1 if prev_actor == "eld" else 0,
                            1 if prev_actor == "hel" else 0])

            this_in.append(self._data[i]["prev_ot_given"])
            this_in.append(self._data[i]["prev_l_given"])
            
            encoded_state = self.encode_state(self._data[i]["prev_hel_ot_belief"], self._data[i]["prev_hel_l_belief"],
                                             self._data[i]["prev_hel_o_belief"])
#
            for j in range(12):
                this_in.append(1 if j+1 == encoded_state else 0)
            

            
            #for j in range(2):
            #    this_in.append(1 if j+1 == self._data[i]["prev_hel_ot_belief"] else 0)
            #for j in range(3):
            #    this_in.append(1 if j+1 == self._data[i]["prev_hel_l_belief"] else 0)
            #for j in range(3):
            #    this_in.append(1 if j+1 == self._data[i]["prev_hel_o_belief"] else 0)


            for j in range(2):
                this_in.append(1 if j+1 == self._data[i]["pt_target"] else 0)
            for j in range(3):
                this_in.append(1 if j+1 == self._data[i]["pt"] else 0)

            for j in range(2):
                this_in.append(1 if j+1 == self._data[i]["ho_target"] else 0)
            for j in range(3):
                this_in.append(1 if j+1 == self._data[i]["ho"] else 0)
            ho_type = self._data[i]["ho_type"]
            this_in.extend([1 if ho_type == "Open" else 0,
                            1 if ho_type == "Takeout" else 0,
                            1 if ho_type == "Touch" else 0,
                            1 if ho_type == "Hold" else 0,
                            1 if ho_type == "Close" else 0])
            
            #for j in range(7):
            #    this_in.append(1 if j+1 == self._data[i]["prev_eld_action"] else 0)
            for j in range(2):
                this_in.append(1 if j+1 == self._data[i]["eld_sees_o"] else 0)
                
            temp = [0]*8 #15, 8
            act = self._data[i]["action"]
            if act:
                temp[act-1] = 1
            
            this_in.extend(temp)
            
            temp1 = [0]*7 #15, 8
            act1 = self._data[i]["prev_eld_action"]
            if act1:
                temp1[act1-1] = 1
            
            this_in.extend(temp1)
            
            temp1 = [0]*13
            if DA[self._data[i]["da"]]:
                temp1[(DA[self._data[i]["da"]])-1] = 1
            this_in.extend(temp1)
        
            temp1 = [0]*13
            if DA[self._data[i]["prev_da"]]:
                temp1[(DA[self._data[i]["prev_da"]])-1] = 1
            this_in.extend(temp1)
            
            #sentences = [self._data[i]["utt"], self._data[i]["prev_utt"]]
            #embedding = self._model.encode(sentences)
            #this_in.extend(embedding[0])
            #this_in.extend(embedding[1])
#
            inputs.append(this_in)
            


        inputs = torch.tensor(inputs, dtype=torch.float)
        da_out = torch.tensor([DA[self._data[i]["eld_da"]] for i in indices])
        action_out = torch.tensor([self._data[i]["eld_action"] if (not self._data[i]["eld_action"]==7) else 1 for i in indices])
        state_out = [self.encode_state(self._data[i]["hel_ot_belief"], self._data[i]["hel_l_belief"], self._data[i]["hel_o_belief"]) for i in indices]
        state_out = torch.tensor(state_out)
        #ot_out = torch.tensor([self._data[i]["hel_ot_belief"] for i in indices])
        #l_out = torch.tensor([self._data[i]["hel_l_belief"] for i in indices])
        #o_out = torch.tensor([self._data[i]["hel_o_belief"] for i in indices])
        
        return (inputs, da_out, action_out, state_out)
        #return (inputs, da_out, ot_out, l_out, o_out, action_out)

In [ ]:
class StateEstimator(nn.Module):
    # hidden_size: size of intermediate layer (between bert+inputs and outputs)
    def __init__(self, hidden_size):
        super(StateEstimator, self).__init__()
        self._hidden_size = hidden_size
        self._dummy = 64
        self._dummy1 = 32
        self._feat_hidden = nn.Linear(54+13+7, self._dummy1)
        #self._feat_hidden = nn.Linear(816, self._dummy1) #425, 816
        self._hidden = nn.Linear(self._dummy1, self._dummy)
        self._hidden1 = nn.Linear(self._dummy, hidden_size)
        self._dropout = nn.Dropout(p=0.2)

        self._da_out = nn.Linear(hidden_size, 14)
        self._action_out = nn.Linear(hidden_size, 7)
        self._state_out = nn.Linear(hidden_size, 13)
        #self._ot_out = nn.Linear(hidden_size, 3)
        #self._l_out = nn.Linear(hidden_size, 4)
        #self._o_out = nn.Linear(hidden_size, 4)
        
    def forward(self, feature_input):
        
        feature_out = self._feat_hidden(feature_input)
        hidden_out = self._dropout(feature_out)
        hidden_out = self._hidden(feature_out)
        hidden_out = self._hidden1(hidden_out)
        #hidden_out = self._dropout(hidden_out)

        to_return = {}
        #to_return["da_out"] = F.relu(self._da_out(hidden_out))
        #to_return["action_out"] = F.relu(self._action_out(hidden_out))
        #to_return["state_out"] = F.relu(self._state_out(hidden_out))
        #to_return["ot_out"] = F.relu(self._ot_out(hidden_out))
        #to_return["l_out"] = F.relu(self._l_out(hidden_out))
        #to_return["o_out"] = F.relu(self._o_out(hidden_out))
        
        #to_return["ot_out"] = self._ot_out(hidden_out)
        #to_return["l_out"] = self._l_out(hidden_out)
        #to_return["o_out"] = self._o_out(hidden_out)
        to_return["da_out"] = self._da_out(hidden_out)
        to_return["action_out"] = self._action_out(hidden_out)
        to_return["state_out"] = self._state_out(hidden_out)
    
        return to_return

In [ ]:
train_dh = DataHandler([data[i] for i in train_idxs]+train_aug, lang_model, tokenizer, shuffle=True)
valid_dh = DataHandler([data[i] for i in valid_idxs]+valid_aug, lang_model, tokenizer)
test_dh = DataHandler([data[i] for i in test_idxs]+test_aug, lang_model, tokenizer)
#test_dh = DataHandler([data[i] for i in test_idxs], lang_model, tokenizer)

In [ ]:
augmented_data = [data[i] for i in range(len(data))]+train_aug+valid_aug+test_aug

In [ ]:
len(augmented_data)

In [ ]:
DA = {"": 0, "Instruct": 1, "Acknowledge": 2, "Query-w": 3, "Query-yn": 4, "Reply-w": 5, "Reply-y": 6, 
      "Reply-n": 7, "Check": 8, "Explain": 9, "Align": 10, "State-y": 11, "State-n": 12, "State": 13}
hel_rule_data = []
for datapoint in augmented_data:
    hel_rule_datapoint = {}
    if datapoint['prev_actor'] == 'eld':
        eld_da = DA[datapoint['prev_da']]
        eld_action = datapoint['prev_eld_action']
    elif datapoint['prev_actor'] == 'hel':
        eld_da = ''
        eld_action = 0
        
    hel_rule_datapoint['eld_da'] = eld_da
    hel_rule_datapoint['eld_action'] = eld_action
    
    # decode helper's state of ot,l,o
    #if datapoint['prev_hel_ot_belief'] == 0:
    if datapoint['hel_ot_belief'] == 0:
        hel_rule_datapoint['ot'] = 0
    elif datapoint['hel_ot_belief'] == 1:
        hel_rule_datapoint['ot'] = 1
    else:
        #print(datapoint['hel_ot_belief'])
        hel_rule_datapoint['ot'] = 2

    if datapoint['hel_l_belief'] == 0:
        hel_rule_datapoint['l'] = 0
    elif datapoint['hel_l_belief'] == 1:
        hel_rule_datapoint['l'] = 1
    else:
        hel_rule_datapoint['l'] = 2
        
    if datapoint['hel_o_belief'] == 0:
        hel_rule_datapoint['o'] = 0
    elif datapoint['hel_o_belief'] == 1:
        hel_rule_datapoint['o'] = 1
    else:
        hel_rule_datapoint['o'] = 2
        
        
    #hel_rule_datapoint['hel_da'] = datapoint['da']
    hel_rule_datapoint['hel_action'] = datapoint['action']
    
    hel_rule_data.append(hel_rule_datapoint)

In [ ]:
with open('../hel_rule_data.json', 'w') as fout:
    json.dump(hel_rule_data, fout)

In [ ]:
print(train_dh._n, valid_dh._n, test_dh._n)

In [ ]:
torch.save(train_dh.get_all_data()[0], 'train_features.pt')
torch.save(train_dh.get_all_data()[1], 'train_da.pt')
torch.save(train_dh.get_all_data()[2], 'train_action.pt')
torch.save(train_dh.get_all_data()[3], 'train_state.pt')

In [ ]:
da_class_weight = {}
for i in range(14):
    da_class_weight[i] = 0

action_class_weight = {}
for i in range(7):
    action_class_weight[i] = 0    
    
for i, (feature_input, da_labels, action_labels, state_labels) in enumerate(train_dh.generate_data()):
    #print(feature_input)
    for j in da_labels:
        for k in range(14):
            if j.item() == k:
                da_class_weight[k] += 1
                
    for j in action_labels:
        for k in range(7):
            if j.item() == k:
                action_class_weight[k] += 1
                           
print('da_class_weight',da_class_weight, '\naction_class_weight',action_class_weight)

In [ ]:
N = sum(da_class_weight.values())
print(N)
weights = []
for i in range(14):
    weight = (da_class_weight[i])/N
    weights.append(weight)

#weights[5]=2
weights

In [ ]:
N = sum(action_class_weight.values())
print(N)
weights_ = []
for i in range(7):
    weight = 1/((N)/(action_class_weight[i]+1))
    weights_.append(weight)

weights_

In [ ]:
model = StateEstimator(64)
model.to(device=device)
optimizer = AdamW(model.parameters(), lr=1e-3) #1e-4 for DA, 1e-3 for SBert
ce = nn.CrossEntropyLoss()

da_weights = torch.FloatTensor(weights).to(device=device)
ce_da = nn.CrossEntropyLoss(weight=da_weights)

action_weights = torch.FloatTensor(weights_).to(device=device)
ce_action = nn.CrossEntropyLoss(weight=action_weights)
best_validation = float("inf")
num_skips = 0 #for early stopping

for epoch in range(1000):
    total_train_loss = 0
    print("Epoch {}, batch ".format(epoch+1), end="")
    
    model.train()
    for i, (feature_input, da_labels, action_labels, state_labels) in enumerate(train_dh.generate_data()):
        print(i+1, end="-")
        model.zero_grad()
        optimizer.zero_grad()
        
        feature_input = feature_input.to(device=device)
        output = model(feature_input)

        loss = torch.zeros(1).to(device=device)
        loss += ce(output["da_out"], da_labels.to(device=device))
        loss += ce_action(output["action_out"], action_labels.to(device=device))
        loss += ce(output["state_out"], state_labels.to(device=device))
        #loss += ce(output["ot_out"], ot_labels.to(device=device))
        #loss += ce(output["l_out"], l_labels.to(device=device))
        #loss += ce(output["o_out"], o_labels.to(device=device))
        loss /= 3
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    print("\nTraining loss:", total_train_loss)

    model.eval()
    feature_input, da_labels, action_labels, state_labels = valid_dh.get_all_data()
    feature_input = feature_input.to(device=device)
    
    with torch.no_grad():
        output = model(feature_input)
        loss = torch.zeros(1).to(device=device)
        loss += ce_da(output["da_out"], da_labels.to(device=device))
        loss += ce_action(output["action_out"], action_labels.to(device=device))
        loss += ce(output["state_out"], state_labels.to(device=device))
        #loss += ce(output["ot_out"], ot_labels.to(device=device))
        #loss += ce(output["l_out"], l_labels.to(device=device))
        #loss += ce(output["o_out"], o_labels.to(device=device))
        loss /= 3
    print("Validation loss:", loss.item())
    if loss.item() < best_validation:
        torch.save(model.state_dict(), "../sim_models/model_30.pt")
        best_validation = loss.item()
        num_skips = 0
    else:
        num_skips += 1
    if num_skips == 15:
        break

In [ ]:
test_dh = DataHandler([data[i] for i in test_idxs]+test_aug, lang_model, tokenizer)
#test_dh = DataHandler([data[i] for i in test_idxs], lang_model, tokenizer)

In [ ]:
DA_REV = {i:da for da,i in DA.items()}


model = StateEstimator(64)
#model = StateEstimator(bert, 128, 64)
 
model.load_state_dict(torch.load("../sim_models/model_29.pt"))
model.to(device=device)
model.eval()

In [ ]:
feature_input, da_labels, action_labels, state_labels = test_dh.get_all_data()

In [ ]:
feature_input = feature_input.to(device=device)

with torch.no_grad():
    output = model(feature_input)

In [ ]:
da_correct = 0
action_correct = 0
state_correct = 0
all_correct = 0

da_pred_all = []
da_target_all = []
action_pred_all = []
action_target_all = []
state_pred_all = []
state_target_all = []

action_score = []
da_score = []

#da_correct = 0
#ot_correct = 0
#l_correct = 0
#o_correct = 0
#action_correct = 0
#all_correct = 0
#state_correct = 0
#non_action_correct = 0
#
#da_pred_all = []
#da_target_all = []
#action_pred_all = []
#action_target_all = []
#ot_pred_all = []
#ot_target_all = []
#l_pred_all = []
#l_target_all = []
#o_pred_all = []
#o_target_all = []
#
#da_score = []
#action_score = []


print("Relu Function")
print("=================")

for i in range(test_dh._n):
    print(i)
    if i < len(test_idxs):
        sample = data[test_idxs[i]]
    else:
        sample = test_aug[i-len(test_idxs)]
    
    #da_pred = torch.argmax(F.softmax(output["da_out"][i], dim=-1)).item()
    #ot_pred = torch.argmax(F.softmax(output["ot_out"][i], dim=-1)).item()
    #l_pred = torch.argmax(F.softmax(output["l_out"][i], dim=-1)).item()
    #o_pred = torch.argmax(F.softmax(output["o_out"][i], dim=-1)).item()
    #action_pred = torch.argmax(F.softmax(output["action_out"][i], dim=-1)).item()
    state_pred = torch.argmax(output["state_out"][i]).item()
    action_pred = torch.argmax(output["action_out"][i]).item()
    da_pred = torch.argmax(output["da_out"][i]).item()
    #ot_pred = torch.argmax(output["ot_out"][i]).item()
    #l_pred = torch.argmax(output["l_out"][i]).item()
    #o_pred = torch.argmax(output["o_out"][i]).item()
    
    state_pred_all.append(state_pred)
    action_pred_all.append(action_pred)
    da_pred_all.append(da_pred)
    #ot_pred_all.append(ot_pred)
    #l_pred_all.append(l_pred)
    #o_pred_all.append(o_pred)

    action_score.append(output["action_out"][i])
    da_score.append(output["da_out"][i])

    state_label = state_labels[i].item()
    action_label = action_labels[i].item()
    da_label = da_labels[i].item()
    #ot_label = ot_labels[i].item()
    #l_label = l_labels[i].item()
    #o_label = o_labels[i].item()

    state_target_all.append(state_label)
    action_target_all.append(action_label)
    da_target_all.append(da_label)
    #ot_target_all.append(ot_label)
    #l_target_all.append(l_label)
    #o_target_all.append(o_label)
    
    if state_pred == state_label: state_correct += 1
    if action_pred == action_label: action_correct += 1
    if da_pred == da_label: da_correct += 1    
        
    if state_pred == state_label and action_pred == action_label and da_pred == da_label:
        all_correct += 1
    #if da_pred == da_label: da_correct += 1
    #if ot_pred == ot_label: ot_correct += 1
    #if l_pred == l_label: l_correct += 1
    #if o_pred == o_label: o_correct += 1
    #if action_pred == action_label: action_correct += 1
    #if ot_pred == ot_label and l_pred == l_label and o_pred == o_label: 
    #    state_correct += 1
    #    if da_pred == da_label:
    #        non_action_correct += 1
    #        if action_pred == action_label:
    #            all_correct += 1
                
    #if da_pred == da_label and action_pred == action_label: all_correct += 1
    print(sample["prev_utt"], "->", sample["utt"])
    #print(sample)
    print('pt:', sample["pt"], 'H-O:', sample['ho'])
    print("Predi:", da_pred, action_pred, state_pred)
    print("Label:", da_label, action_label, state_label)
    #print("Predi:", action_pred, DA_REV[da_pred], ot_pred, l_pred, o_pred)
    #print("Label:", action_label, DA_REV[da_label], ot_label, l_label, o_label)
    print("-----------------")
print("=================")
print("All:", all_correct, "/", test_dh._n, "=", all_correct/test_dh._n)
print("DA:", da_correct, "/", test_dh._n, "=", da_correct/test_dh._n)
print("Action:", action_correct, "/", test_dh._n, "=", action_correct/test_dh._n)
print("State:", state_correct, "/", test_dh._n, "=", state_correct/test_dh._n)

#print("All:", all_correct, "/", test_dh._n, "=", all_correct/test_dh._n)
#print("Action:", action_correct, "/", test_dh._n, "=", action_correct/test_dh._n)
#print("Non-Action:", non_action_correct, "/", test_dh._n, "=", non_action_correct/test_dh._n)
#print("DA:", da_correct, "/", test_dh._n, "=", da_correct/test_dh._n)
#print("State:", state_correct, "/", test_dh._n, "=", state_correct/test_dh._n)
#print("Ot:", ot_correct, "/", test_dh._n, "=", ot_correct/test_dh._n)
#print("L:", l_correct, "/", test_dh._n, "=", l_correct/test_dh._n)
#print("O:", o_correct, "/", test_dh._n, "=", o_correct/test_dh._n)


confmat_da = ConfusionMatrix(num_classes=14)
confusion_da = confmat_da(torch.tensor(da_pred_all), torch.tensor(da_target_all))
#
#confmat_ot = ConfusionMatrix(num_classes=3)
#confusion_ot = confmat_ot(torch.tensor(ot_pred_all), torch.tensor(ot_target_all))
#
#confmat_o = ConfusionMatrix(num_classes=4)
#confusion_o = confmat_o(torch.tensor(o_pred_all), torch.tensor(o_target_all))
#
#confmat_l = ConfusionMatrix(num_classes=4)
#confusion_l = confmat_l(torch.tensor(l_pred_all), torch.tensor(l_target_all))
#
confmat_action = ConfusionMatrix(num_classes=7)
confusion_action = confmat_action(torch.tensor(action_pred_all), torch.tensor(action_target_all))
#
print("DA Confusion=\n", confusion_da)
count = 0
for each in da_score:
    print("da_score_vector (pred {},label {})=\n".format(da_pred_all[count], da_target_all[count]), each)
    count+=1
#    
##print("da_preds_vector=\n", da_pred_all)
##print("da_label_vector=\n", da_target_all)
#
#
print("Action Confusion=\n", confusion_action)
count = 0
for each in action_score:
    print("action_score_vector (pred {},label {})=\n".format(action_pred_all[count], action_target_all[count]), each)
    count+=1

In [ ]:
DA_REV = {i:da for da,i in DA.items()}


model = StateEstimator(64)
#model = StateEstimator(bert, 128, 64)
 
model.load_state_dict(torch.load("../sim_models/model_19.pt"))
model.to(device=device)
model.eval()

feature_input, da_labels, action_labels, ot_labels, l_labels, o_labels = test_dh.get_all_data()
feature_input = feature_input.to(device=device)

with torch.no_grad():
    output = model(feature_input)


#da_correct = 0
#action_correct = 0
#state_correct = 0
#all_correct = 0
#
#da_pred_all = []
#da_target_all = []
#action_pred_all = []
#action_target_all = []
#state_pred_all = []
#state_target_all = []
#
#action_score = []
#da_score = []

da_correct = 0
ot_correct = 0
l_correct = 0
o_correct = 0
action_correct = 0
all_correct = 0
state_correct = 0
non_action_correct = 0

da_pred_all = []
da_target_all = []
action_pred_all = []
action_target_all = []
ot_pred_all = []
ot_target_all = []
l_pred_all = []
l_target_all = []
o_pred_all = []
o_target_all = []

da_score = []
action_score = []


print("Relu Function")
print("=================")

for i in range(test_dh._n):
    print(i)
    if i < len(test_idxs):
        sample = data[test_idxs[i]]
    else:
        sample = test_aug[i-len(test_idxs)]
    
    #da_pred = torch.argmax(F.softmax(output["da_out"][i], dim=-1)).item()
    #ot_pred = torch.argmax(F.softmax(output["ot_out"][i], dim=-1)).item()
    #l_pred = torch.argmax(F.softmax(output["l_out"][i], dim=-1)).item()
    #o_pred = torch.argmax(F.softmax(output["o_out"][i], dim=-1)).item()
    #action_pred = torch.argmax(F.softmax(output["action_out"][i], dim=-1)).item()
    #state_pred = torch.argmax(output["state_out"][i]).item()
    action_pred = torch.argmax(output["action_out"][i]).item()
    da_pred = torch.argmax(output["da_out"][i]).item()
    ot_pred = torch.argmax(output["ot_out"][i]).item()
    l_pred = torch.argmax(output["l_out"][i]).item()
    o_pred = torch.argmax(output["o_out"][i]).item()
    
    #state_pred_all.append(state_pred)
    action_pred_all.append(action_pred)
    da_pred_all.append(da_pred)
    ot_pred_all.append(ot_pred)
    l_pred_all.append(l_pred)
    o_pred_all.append(o_pred)

    action_score.append(output["action_out"][i])
    da_score.append(output["da_out"][i])

    #state_label = state_labels[i].item()
    action_label = action_labels[i].item()
    da_label = da_labels[i].item()
    ot_label = ot_labels[i].item()
    l_label = l_labels[i].item()
    o_label = o_labels[i].item()

    #state_target_all.append(state_label)
    action_target_all.append(action_label)
    da_target_all.append(da_label)
    ot_target_all.append(ot_label)
    l_target_all.append(l_label)
    o_target_all.append(o_label)
    
    #if state_pred == state_label: state_correct += 1
    #if action_pred == action_label: action_correct += 1
    #if da_pred == da_label: da_correct += 1    
    #    
    #if state_pred == state_label and action_pred == action_label and da_pred == da_label:
    #    all_correct += 1
    if da_pred == da_label: da_correct += 1
    if ot_pred == ot_label: ot_correct += 1
    if l_pred == l_label: l_correct += 1
    if o_pred == o_label: o_correct += 1
    if action_pred == action_label: action_correct += 1
    if ot_pred == ot_label and l_pred == l_label and o_pred == o_label: 
        state_correct += 1
        if da_pred == da_label:
            #non_action_correct += 1
            if action_pred == action_label:
                all_correct += 1
                
    #if da_pred == da_label and action_pred == action_label: all_correct += 1
    print(sample["prev_utt"], "->", sample["utt"])
    print('pt:', sample["pt"], 'H-O:', sample['ho'])
    #print("Predi:", da_pred, action_pred, state_pred)
    #print("Label:", da_label, action_label, state_label)
    print("Predi:", action_pred, DA_REV[da_pred], ot_pred, l_pred, o_pred)
    print("Label:", action_label, DA_REV[da_label], ot_label, l_label, o_label)
    print("-----------------")
print("=================")
#print("All:", all_correct, "/", test_dh._n, "=", all_correct/test_dh._n)
#print("DA:", da_correct, "/", test_dh._n, "=", da_correct/test_dh._n)
#print("Action:", action_correct, "/", test_dh._n, "=", action_correct/test_dh._n)
#print("State:", state_correct, "/", test_dh._n, "=", state_correct/test_dh._n)

print("All:", all_correct, "/", test_dh._n, "=", all_correct/test_dh._n)
print("Action:", action_correct, "/", test_dh._n, "=", action_correct/test_dh._n)
#print("Non-Action:", non_action_correct, "/", test_dh._n, "=", non_action_correct/test_dh._n)
print("DA:", da_correct, "/", test_dh._n, "=", da_correct/test_dh._n)
print("State:", state_correct, "/", test_dh._n, "=", state_correct/test_dh._n)
print("Ot:", ot_correct, "/", test_dh._n, "=", ot_correct/test_dh._n)
print("L:", l_correct, "/", test_dh._n, "=", l_correct/test_dh._n)
print("O:", o_correct, "/", test_dh._n, "=", o_correct/test_dh._n)


confmat_da = ConfusionMatrix(num_classes=14)
confusion_da = confmat_da(torch.tensor(da_pred_all), torch.tensor(da_target_all))

confmat_ot = ConfusionMatrix(num_classes=3)
confusion_ot = confmat_ot(torch.tensor(ot_pred_all), torch.tensor(ot_target_all))

confmat_o = ConfusionMatrix(num_classes=4)
confusion_o = confmat_o(torch.tensor(o_pred_all), torch.tensor(o_target_all))

confmat_l = ConfusionMatrix(num_classes=4)
confusion_l = confmat_l(torch.tensor(l_pred_all), torch.tensor(l_target_all))

confmat_action = ConfusionMatrix(num_classes=7)
confusion_action = confmat_action(torch.tensor(action_pred_all), torch.tensor(action_target_all))

print("DA Confusion=\n", confusion_da)
count = 0
for each in da_score:
    print("da_score_vector (pred {},label {})=\n".format(da_pred_all[count], da_target_all[count]), each)
    count+=1
    
#print("da_preds_vector=\n", da_pred_all)
#print("da_label_vector=\n", da_target_all)


print("Action Confusion=\n", confusion_action)
count = 0
for each in action_score:
    print("action_score_vector (pred {},label {})=\n".format(action_pred_all[count], action_target_all[count]), each)
    count+=1